# Import all files and process to separate file based on annotation

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mne.preprocessing import compute_current_source_density
import seaborn as sns
from mne.decoding import PSDEstimator
import matplotlib
# matplotlib.use('Qt5Agg')
import os
import mne

## File 1-10

In [5]:
import os
import shutil
import mne

# Load and Parse Data
def load_and_parse_data(P_folder):
    all_edf_file_paths = [os.path.join(P_folder, f) for f in os.listdir(P_folder) if f.endswith('.edf')]
    summary_file_path = [os.path.join(P_folder, f) for f in os.listdir(P_folder) if f.endswith('.txt')][0]
    
    with open(summary_file_path, 'r') as file:
        summary_content = file.readlines()

    seizure_times = parse_seizure_times(summary_content)
    return all_edf_file_paths, seizure_times

def parse_seizure_times(summary_content):
    seizure_times = {}
    current_file = None
    summary_iterator = iter(summary_content)  # Convert list to iterator

    for line in summary_iterator:
        if 'File Name' in line:
            current_file = line.split(':')[-1].strip()
            seizure_times[current_file] = []
        elif 'Seizure' in line and 'Start Time' in line:
            start_time = int(line.split(':')[-1].strip().split(' ')[0])
            end_time_line = next(summary_iterator)
            if 'End Time' in end_time_line:
                end_time = int(end_time_line.split(':')[-1].strip().split(' ')[0])
                seizure_times[current_file].append((start_time, end_time))
    
    return seizure_times



# Annotate EEG Data
def annotate_eeg_data(raw, seizure_times, preictal_duration=30*60, postictal_duration=10*60, buffer_duration=30*60):
    file_name = os.path.basename(raw.filenames[0])

    if file_name in seizure_times:
        last_end = 0
        for start, end in seizure_times[file_name]:
            if start - preictal_duration - buffer_duration > 0 and last_end < start - preictal_duration - buffer_duration:
                raw.annotations.append(onset=start - preictal_duration - buffer_duration, duration=buffer_duration, description='buffer_before_preictal')
            
            preictal_start = max(last_end, start - preictal_duration)
            preictal_duration_actual = start - preictal_start
            if preictal_duration_actual > 0:  # Ensure positive duration
                raw.annotations.append(onset=preictal_start, duration=preictal_duration_actual, description='preictal')
            
            duration = end - start
            raw.annotations.append(onset=start, duration=duration, description='ictal')
            
            postictal_start = end
            postictal_duration_actual = min(postictal_duration, raw.times[-1] - postictal_start)
            if postictal_duration_actual > 0:  # Ensure positive duration
                raw.annotations.append(onset=postictal_start, duration=postictal_duration_actual, description='postictal')

            last_end = postictal_start + postictal_duration_actual + buffer_duration
        
        if last_end < raw.times[-1]:
            interictal_duration = raw.times[-1] - last_end
            if interictal_duration > 0:  # Ensure positive duration
                raw.annotations.append(onset=last_end, duration=interictal_duration, description='interictal')
    else:
        raw.annotations.append(onset=0, duration=raw.times[-1], description='interictal')

    return raw


# Preprocess the EEG Data
def preprocess_eeg_data(raw):
    raw = ChannelSelection(raw)
    raw.filter(l_freq=0.5, h_freq=100)
    raw_notch = raw.copy().notch_filter(freqs=[60], notch_widths=6)
    return raw_notch

def ChannelSelection(signal):

    # List of channels to retain
    desired_channels = [
        'FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1',
        'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1',
        'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2',
        'FP2-F8', 'F8-T8', 'T8-P8-0','T8-P8', 'P8-O2',
        'FZ-CZ', 'CZ-PZ', 'T7-FT9',
        'FT9-FT10', 'FT10-T8'
    ]

    # Check existing channels in the signal
    available_channels = signal.info['ch_names']

    # Filter desired channels based on availability in the signal
    selected_channels = [ch for ch in desired_channels if ch in available_channels]

    # Pick only the selected channels
    signal.pick_channels(selected_channels)

    # Rename channel if needed
    if 'T8-P8-0' in signal.info['ch_names']:
        channel_mapping = {'T8-P8-0': 'T8-P8'}
        mne.rename_channels(signal.info, channel_mapping)

    return signal

        

            

# Function to crop and save annotations
def crop_and_save_annotations(raw, subject_folder, file_name):
    # Initialize a counter dictionary to keep track of file numbers for each description
    description_count = {}

    for ann in raw.annotations:
        description = ann['description']
        onset = ann['onset']
        duration = ann['duration']
        
        # Increment the counter for the description
        if description in description_count:
            description_count[description] += 1
        else:
            description_count[description] = 1

        # Crop the raw data based on the annotation's onset and duration
        cropped_raw = raw.copy().crop(tmin=onset, tmax=onset + duration)
        cropped_raw.set_meas_date(None)
        
        # Create output folder for this annotation type if it doesn't exist
        annotation_folder = os.path.join(subject_folder, description)
        if not os.path.exists(annotation_folder):
            os.makedirs(annotation_folder)
        
        # Construct a unique filename using the counter
        output_file = os.path.join(annotation_folder, f"{file_name}_{description}_{description_count[description]}_raw.fif")
        
        # Save the cropped data
        cropped_raw.save(output_file, overwrite=True)
        print(f"Saved {description} segment to {output_file}")



# Process and Annotate Data, then Save Based on Annotation Type
def process_and_save(P_folder):
    all_edf_file_paths, seizure_times = load_and_parse_data(P_folder)
    
    
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>{seizure_times}")
    for edf_file in all_edf_file_paths:
        print(f"Processing {edf_file}...")
        raw = mne.io.read_raw_edf(edf_file, preload=True)

        
        
        raw = annotate_eeg_data(raw, seizure_times)
        raw = preprocess_eeg_data(raw)
        # raw.plot()
        raw.set_meas_date(None)   # Setting meas_date to None
        

        # raw.plot()
        file_name = os.path.basename(edf_file)
        subject_name = file_name.split('_')[0]
        subject_folder = os.path.join(P_folder, subject_name)

        # Check if the file contains seizures
        if file_name in seizure_times and len(seizure_times[file_name]) > 0:
            # Process and save ictal, preictal, postictal data
            crop_and_save_annotations(raw, subject_folder, file_name)
            # raw.plot()
        else:
            # Move to normal folder
            normal_folder = os.path.join(subject_folder, 'interictal')
            if not os.path.exists(normal_folder):
                os.makedirs(normal_folder)
            raw.save(os.path.join(normal_folder, file_name.replace('.edf', '_normal_raw.fif')), overwrite=True)

In [6]:
common = ['chb01',
 'chb02',
 'chb03',
 'chb04',
 'chb05',
 'chb06',
 'chb07',
 'chb08',
 'chb09',
 'chb10']
# folder = 'D:\\archive\\chb-mit-scalp-eeg-database-1.0.0'
folder = r'E:\New folder'
for i in [os.path.join(folder,j) for j in common]:
    print(i)
    process_and_save(i)

E:\New folder\chb01
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>{'chb01_01.edf': [], 'chb01_02.edf': [], 'chb01_03.edf': [(2996, 3036)], 'chb01_04.edf': [(1467, 1494)], 'chb01_05.edf': [], 'chb01_06.edf': [], 'chb01_07.edf': [], 'chb01_08.edf': [], 'chb01_09.edf': [], 'chb01_10.edf': [], 'chb01_11.edf': [], 'chb01_12.edf': [], 'chb01_13.edf': [], 'chb01_14.edf': [], 'chb01_15.edf': [(1732, 1772)], 'chb01_16.edf': [(1015, 1066)], 'chb01_17.edf': [], 'chb01_18.edf': [(1720, 1810)], 'chb01_19.edf': [], 'chb01_20.edf': [], 'chb01_21.edf': [(327, 420)], 'chb01_22.edf': [], 'chb01_23.edf': [], 'chb01_24.edf': [], 'chb01_25.edf': [], 'chb01_26.edf': [(1862, 1963)], 'chb01_27.edf': [], 'chb01_29.edf': [], 'chb01_30.edf': [], 'chb01_31.edf': [], 'chb01_32.edf': [], 'chb01_33.edf': [], 'chb01_34.edf': [], 'chb01_36.edf': [], 'chb01_37.edf': [], 'chb01_38.edf': [], 'chb01_39.edf': [], 'chb01_40.edf': [], 'chb01_41.edf': [], 'chb01_42.edf': [], 'chb01_43.edf': [], 'ch

C:\Users\pc\AppData\Local\Temp\ipykernel_5444\3140418589.py:152: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 1691 samples (6.605 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.7s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 56 - 64 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 56.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Upper passband edge: 63.50 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 63.75 Hz)
- Filter length: 1691 samples (6.605 s)



...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 1691 samples (6.605 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.2s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 56 - 64 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 56.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Upper passband edge: 63.50 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 63.75 Hz)
- Filter length: 1691 samples (6.605 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.2s


Writing E:\New folder\chb10\chb10\preictal\chb10_89.edf_preictal_1_raw.fif
Closing E:\New folder\chb10\chb10\preictal\chb10_89.edf_preictal_1_raw.fif
[done]
Saved preictal segment to E:\New folder\chb10\chb10\preictal\chb10_89.edf_preictal_1_raw.fif
Writing E:\New folder\chb10\chb10\ictal\chb10_89.edf_ictal_1_raw.fif
Closing E:\New folder\chb10\chb10\ictal\chb10_89.edf_ictal_1_raw.fif
[done]
Saved ictal segment to E:\New folder\chb10\chb10\ictal\chb10_89.edf_ictal_1_raw.fif
Writing E:\New folder\chb10\chb10\postictal\chb10_89.edf_postictal_1_raw.fif
Closing E:\New folder\chb10\chb10\postictal\chb10_89.edf_postictal_1_raw.fif
[done]
Saved postictal segment to E:\New folder\chb10\chb10\postictal\chb10_89.edf_postictal_1_raw.fif
Writing E:\New folder\chb10\chb10\interictal\chb10_89.edf_interictal_1_raw.fif
Closing E:\New folder\chb10\chb10\interictal\chb10_89.edf_interictal_1_raw.fif
[done]
Saved interictal segment to E:\New folder\chb10\chb10\interictal\chb10_89.edf_interictal_1_raw.fif


## File 11

In [5]:
paitent11 = "D:\\archive\\chb-mit-scalp-eeg-database-1.0.0\\chb11"
process_and_save(paitent11)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>{'chb11_01.edf': [], 'chb11_02.edf': [], 'chb11_03.edf': [], 'chb11_04.edf': [], 'chb11_05.edf': [], 'chb11_06.edf': [], 'chb11_07.edf': [], 'chb11_08.edf': [], 'chb11_09.edf': [], 'chb11_10.edf': [], 'chb11_11.edf': [], 'chb11_12.edf': [], 'chb11_13.edf': [], 'chb11_14.edf': [], 'chb11_15.edf': [], 'chb11_16.edf': [], 'chb11_17.edf': [], 'chb11_18.edf': [], 'chb11_19.edf': [], 'chb11_24.edf': [], 'chb11_25.edf': [], 'chb11_26.edf': [], 'chb11_27.edf': [], 'chb11_53.edf': [], 'chb11_54.edf': [], 'chb11_55.edf': [], 'chb11_56.edf': [], 'chb11_58.edf': [], 'chb11_60.edf': [], 'chb11_61.edf': [], 'chb11_62.edf': [], 'chb11_63.edf': [], 'chb11_82.edf': [(298, 320)], 'chb11_92.edf': [(2695, 2727)], 'chb11_99.edf': [(1454, 2206)]}
Processing D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11_01.edf...
Extracting EDF parameters from D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11_01.edf...
EDF file detected

C:\Users\ACER\AppData\Local\Temp\ipykernel_9460\1738164950.py:135: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 56 - 64 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 56.50
- Lowe

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Writing D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11\interictal\chb11_01_normal_raw.fif


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 56 - 64 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 56.50
- Lowe

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


duration =============================================================================  1454.0
Writing D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11\preictal\chb11_99.edf_preictal_raw.fif


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Closing D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11\preictal\chb11_99.edf_preictal_raw.fif
[done]
Saved preictal segment to D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11\preictal\chb11_99.edf_preictal_raw.fif
duration =============================================================================  752.0
Writing D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11\ictal\chb11_99.edf_ictal_raw.fif
Closing D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11\ictal\chb11_99.edf_ictal_raw.fif
[done]
Saved ictal segment to D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11\ictal\chb11_99.edf_ictal_raw.fif
duration =============================================================================  600.0
Writing D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11\postictal\chb11_99.edf_postictal_raw.fif
Closing D:\archive\chb-mit-scalp-eeg-database-1.0.0\chb11\chb11\postictal\chb11_99.edf_postictal_raw.fif
[done]
Saved postictal segment to D:\archive\chb-mit-scalp

## File 12

##### not considered yet: 12-27,28,29

In [1]:
# Example usage
P_folder = 'D:\\archive\\chb-mit-scalp-eeg-database-1.0.0\\chb12'
all_edf_file_paths, seizure_times = load_and_parse_data(P_folder)

NameError: name 'load_and_parse_data' is not defined

In [ ]:
run_process = all_edf_file_paths[24]
raw = mne.io.read_raw_edf(run_process, preload=True)
raw = annotate_eeg_data(raw, seizure_times)
raw.plot()

In [ ]:
channels = [
        'FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1',
        'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1',
        'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2',
        'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2',
        'FZ-CZ', 'CZ-PZ', 'T7-FT9',
        'FT9-FT10', 'FT10-T8',
    ]

raw.pick_channels(channels)
channel_mapping = {'T8-P8-0': 'T8-P8'}
mne.rename_channels(raw.info, channel_mapping)

# raw.plot()

In [ ]:
process_and_save(run_process, raw, seizure_times)

# File formating

In [8]:
import os 

In [9]:
import os
import shutil

def create_destination_structure(dest_root):
    """Create the destination folder structure."""
    subfolders = ['ictal', 'interictal', 'postictal', 'preictal']
    for subfolder in subfolders:
        path = os.path.join(dest_root, subfolder)
        if not os.path.exists(path):
            os.makedirs(path)

def copy_and_rename_files(src_folder, dest_folder, patient_id):
    """Copy and rename .fif files from the source to the destination structure."""
    for root, _, files in os.walk(src_folder):
        for file in files:
            if file.endswith('.fif'):
                file_type = os.path.basename(root)
                if file_type in ['ictal', 'interictal', 'postictal', 'preictal']:
                    dest_subfolder = os.path.join(dest_folder, file_type, patient_id)
                    if not os.path.exists(dest_subfolder):
                        os.makedirs(dest_subfolder)
                    
                    src_file = os.path.join(root, file)
                    dest_file = os.path.join(dest_subfolder, file)
                    shutil.copy2(src_file, dest_file)
                    print(f"Copied {file} to {dest_file}")

def process_chbmit_dataset(main_folder, dest_root):
    """Process the CHB-MIT dataset."""
    for patient_folder in os.listdir(main_folder):
        patient_path = os.path.join(main_folder, patient_folder)
        if os.path.isdir(patient_path) and patient_folder.startswith('chb'):
            patient_inner_folder = os.path.join(patient_path, patient_folder)
            copy_and_rename_files(patient_inner_folder, dest_root, patient_folder)

# Define the paths
main_folder = r'E:\New folder'
dest_root = r'D:\CHB_MIT_crop_data'

# Create the necessary directory structure in D: drive
create_destination_structure(dest_root)

# Process and copy files from the main dataset to the new structure
process_chbmit_dataset(main_folder, dest_root)

Copied chb01_03.edf_ictal_1_raw.fif to D:\CHB_MIT_crop_data\ictal\chb01\chb01_03.edf_ictal_1_raw.fif
Copied chb01_04.edf_ictal_1_raw.fif to D:\CHB_MIT_crop_data\ictal\chb01\chb01_04.edf_ictal_1_raw.fif
Copied chb01_15.edf_ictal_1_raw.fif to D:\CHB_MIT_crop_data\ictal\chb01\chb01_15.edf_ictal_1_raw.fif
Copied chb01_16.edf_ictal_1_raw.fif to D:\CHB_MIT_crop_data\ictal\chb01\chb01_16.edf_ictal_1_raw.fif
Copied chb01_18.edf_ictal_1_raw.fif to D:\CHB_MIT_crop_data\ictal\chb01\chb01_18.edf_ictal_1_raw.fif
Copied chb01_21.edf_ictal_1_raw.fif to D:\CHB_MIT_crop_data\ictal\chb01\chb01_21.edf_ictal_1_raw.fif
Copied chb01_26.edf_ictal_1_raw.fif to D:\CHB_MIT_crop_data\ictal\chb01\chb01_26.edf_ictal_1_raw.fif
Copied chb01_01_normal_raw.fif to D:\CHB_MIT_crop_data\interictal\chb01\chb01_01_normal_raw.fif
Copied chb01_02_normal_raw.fif to D:\CHB_MIT_crop_data\interictal\chb01\chb01_02_normal_raw.fif
Copied chb01_05_normal_raw.fif to D:\CHB_MIT_crop_data\interictal\chb01\chb01_05_normal_raw.fif
Copie